# Лабораторная работа 1. Введение в машинное обучение. Обучение с учителем. Задача регрессии

<b>Традиционное предупреждение для всех лабораторных работ:</b> перед обучением моделей необходимо выполнить предварительную обработку данных, которая <b>обязательно</b> включает в себя:
- заполнение пропущенных значений (рекомедуется логика заполнения пропусков на основе типа данных, которая использовалась в РГР по Практикуму);
- преобразование категориальных признаков в числовые (используйте one-hot кодирование или map; используйте знания с Практикума).

Предобработка может включать в себя другие действия, но выполнение описанных выше действий обязательно.

Сделайте это один раз и сохраните в отдельный csv файл, а потом его используйте.

<b>Выполните следующие задания:</b>
- загрузите датасет для регрессии, выделите целевой признак и предикторы, разбейте данные на обучающую и тестовую выборку;
- решите задачу регрессии на ваших данных с использованием моделей sklearn (линейная регрессия + L1, L2), для моделей с регуляризациями подберите гиперпараметр;
- решите задачу регрессии на ваших данных с использованием моделей sklearn (полиномиальная регрессия + L1, L2), для моделей с регуляризациями подберите гиперпараметр;
- вычислите значения метрик $R^2$, MAE, MSE, RMSE, MAPE для всех обученных моделей; выберите лучшую модель;
- самостоятельно реализуйте (желательно в виде класса) модель линейной регрессии с регуляризацией (можете выбрать L1 или L2);
- самостоятельно реализуйте вычисление всех используемых метрик (в виде функций, принимающих два аргумента);
- обучите вашу модель линейной регрессии на ваших данных; оцените качество с помощью реализованных вами метрик.

In [903]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from math import sqrt
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt

In [904]:
data= pd.read_csv("../data/trip_duration_task.csv")
data = data.head(170000)
data.drop(['id','pickup_datetime', 'dropoff_datetime', 'vendor_id' ], axis=1, inplace=True)
data.drop_duplicates(inplace=True)
data.dropna(inplace=True)
data.corr()

,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
passenger_count,1.000000,0.007919,-0.008300,0.001692,-0.005364,0.007716
pickup_longitude,0.007919,1.000000,-0.147094,0.273100,0.042711,0.052203
pickup_latitude,-0.008300,-0.147094,1.000000,0.053645,0.430459,-0.037986
dropoff_longitude,0.001692,0.273100,0.053645,1.000000,0.134767,0.029849
dropoff_latitude,-0.005364,0.042711,0.430459,0.134767,1.000000,-0.021712
trip_duration,0.007716,0.052203,-0.037986,0.029849,-0.021712,1.000000


In [905]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 169540 entries, 0 to 169999
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   passenger_count    169540 non-null  int64  
 1   pickup_longitude   169540 non-null  float64
 2   pickup_latitude    169540 non-null  float64
 3   dropoff_longitude  169540 non-null  float64
 4   dropoff_latitude   169540 non-null  float64
 5   trip_duration      169540 non-null  int64  
dtypes: float64(4), int64(2)
memory usage: 9.1 MB


Предобработка данных

1) Убрал строки с null-value в столбце "vendor_id"
2) Заполнил строки с null-value в столбце "pickup_latitude"

In [906]:
y = data['trip_duration']
X = data.drop(['trip_duration'], axis=1)



In [907]:
y

0          400
1         1100
2         1635
3         1141
4          848
          ... 
169995     183
169996     534
169997    1221
169998     551
169999    1589
Name: trip_duration, Length: 169540, dtype: int64

In [908]:
X

,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,1,-73.953918,40.778873,-73.963875,40.771164
1,2,-73.988312,40.731743,-73.994751,40.694931
2,2,-73.997314,40.721458,-73.948029,40.774918
3,6,-73.961670,40.759720,-73.956779,40.780628
4,1,-74.017120,40.708469,-73.988182,40.740631
...,...,...,...,...,...
169995,1,-73.999718,40.679214,-73.994370,40.686272
169996,1,-73.978210,40.752140,-73.984535,40.736977
169997,2,-73.991982,40.738674,-74.009193,40.710205
169998,1,-73.970970,40.798481,-73.988731,40.768250


In [909]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [910]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((135632, 5), (135632,), (33908, 5), (33908,))

In [911]:
lr = LinearRegression().fit(X_train, y_train)

In [912]:
lr.predict(X_test)
list_column_weight = list(zip(X.columns,lr.coef_))
list_column_weight


[('passenger_count', 28.723625335972958),
 ('pickup_longitude', 6302.408102997084),
 ('pickup_latitude', -5607.656698280493),
 ('dropoff_longitude', 3380.82081113982),
 ('dropoff_latitude', -2362.3815384918485)]

In [913]:
y_pred = lr.predict(X_test)
y_pred.shape

(33908,)

In [914]:
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {r2_score(y_test, y_pred)}')

MAE: 574.4135718583129
MSE: 10357957.384636324
RMSE: 3218.378067386789
MAPE: 1.205243795333324
R^2: 0.011374017295675443


In [915]:
len(lr.coef_)
lr.coef_

array([   28.72362534,  6302.408103  , -5607.65669828,  3380.82081114,
       -2362.38153849])

In [916]:
ridge = Ridge(alpha=0.9).fit(X_train, y_train)
y_pred = ridge.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {r2_score(y_test, y_pred)}')
ridge.coef_

MAE: 574.4153304863943
MSE: 10357830.039774314
RMSE: 3218.3582833137634
MAPE: 1.2052450453771788
R^2: 0.011386171858073446


array([   28.73890982,  6281.83335309, -5565.18709867,  3367.95389361,
       -2360.57653096])

In [917]:
parameters = {'alpha': np.arange(0, 1, 0.1)}
ridge_optimal = GridSearchCV(Ridge(), parameters).fit(X_train, y_train)
ridge_optimal.best_params_

{'alpha': 0.9}

In [918]:
lasso = Lasso(alpha=0.9)
lasso.fit(X_train, y_train)
y_pred = lasso.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {r2_score(y_test, y_pred)}')
ridge.coef_

MAE: 576.6092185153
MSE: 10358088.315401288
RMSE: 3218.398408432568
MAPE: 1.2058148177778092
R^2: 0.011361520473051834


array([   28.73890982,  6281.83335309, -5565.18709867,  3367.95389361,
       -2360.57653096])

In [919]:
polynomialFeatures = PolynomialFeatures(2)
X = X.values.reshape(-1, 5)
y = y.values.reshape(-1, 1)


In [920]:
lr = LinearRegression().fit(X, y)
lr.coef_

array([[   30.05775688,  6285.77760407, -5219.24617295,  3304.7476271 ,
        -2605.73276649]])

In [921]:
X_pf = polynomialFeatures.fit_transform(X)